In [1]:
#!pip install xgboost

In [2]:
import os
import pandas as pd
import numpy as np

import psycopg2
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy import text

import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

from datetime import timedelta

In [3]:
def conexion_RDS():

    with open("credentials/bbdd.json") as f:
        bbdd = json.load(f)
    
    host=bbdd["host"]
    database=bbdd["database"]
    user=bbdd["user"]
    password=bbdd["password"]

    engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")
    #Bloque de test - cierra automáticamente la conexión
    with engine.connect() as conn:
        rows = conn.execute(text("SELECT 1")).all()
        df = pd.read_sql_table(table_name = "demanda_electrica", con = conn)
        print(df.head())

    connection = engine.connect()
    df = pd.read_sql_table(table_name = "demanda_electrica", con = connection)
    connection.close()
    engine.dispose()

    return df
    


In [4]:
if not os.path.isfile("demanda.csv"):

    df=conexion_RDS()
    df.to_csv("demanda.csv",index=False)

else:
    df=pd.read_csv("demanda.csv")
    print(df.head(20))

        value                       datetime          datetime_utc  \
0   3187298.0  2014-01-01T00:00:00.000+01:00  2013-12-31T23:00:00Z   
1   3916726.0  2014-01-02T00:00:00.000+01:00  2014-01-01T23:00:00Z   
2   3957472.0  2014-01-03T00:00:00.000+01:00  2014-01-02T23:00:00Z   
3   3726719.0  2014-01-04T00:00:00.000+01:00  2014-01-03T23:00:00Z   
4   3511386.0  2014-01-05T00:00:00.000+01:00  2014-01-04T23:00:00Z   
5   3309795.0  2014-01-06T00:00:00.000+01:00  2014-01-05T23:00:00Z   
6   4153137.0  2014-01-07T00:00:00.000+01:00  2014-01-06T23:00:00Z   
7   4255484.0  2014-01-08T00:00:00.000+01:00  2014-01-07T23:00:00Z   
8   4298819.0  2014-01-09T00:00:00.000+01:00  2014-01-08T23:00:00Z   
9   4292097.0  2014-01-10T00:00:00.000+01:00  2014-01-09T23:00:00Z   
10  3867730.0  2014-01-11T00:00:00.000+01:00  2014-01-10T23:00:00Z   
11  3601448.0  2014-01-12T00:00:00.000+01:00  2014-01-11T23:00:00Z   
12  4334742.0  2014-01-13T00:00:00.000+01:00  2014-01-12T23:00:00Z   
13  4433714.0  2014-

In [5]:
df_backup = df.copy()

#df = df_backup

In [6]:
df_clean = df.copy()

In [7]:
df_clean.isnull().sum()

value           0
datetime        0
datetime_utc    0
tz_time         0
geo_id          0
geo_name        0
dtype: int64

In [8]:
df_clean["geo_name"].value_counts(dropna=False)

geo_name
Península    4277
Name: count, dtype: int64

In [9]:
df_clean["geo_id"].value_counts(dropna=False)

geo_id
8741    4277
Name: count, dtype: int64

In [10]:
df_clean["value"]

0       3187298.0
1       3916726.0
2       3957472.0
3       3726719.0
4       3511386.0
          ...    
4272    7719284.0
4273    6931088.0
4274    6490902.0
4275    3386155.0
4276    8521577.0
Name: value, Length: 4277, dtype: float64

In [11]:
df_clean["datetime"]

0       2014-01-01T00:00:00.000+01:00
1       2014-01-02T00:00:00.000+01:00
2       2014-01-03T00:00:00.000+01:00
3       2014-01-04T00:00:00.000+01:00
4       2014-01-05T00:00:00.000+01:00
                    ...              
4272    2025-09-12T00:00:00.000+02:00
4273    2025-09-13T00:00:00.000+02:00
4274    2025-09-14T00:00:00.000+02:00
4275    2025-09-15T00:00:00.000+02:00
4276    2025-09-16T00:00:00.000+02:00
Name: datetime, Length: 4277, dtype: object

In [12]:
df_clean["datetime_utc"]

0       2013-12-31T23:00:00Z
1       2014-01-01T23:00:00Z
2       2014-01-02T23:00:00Z
3       2014-01-03T23:00:00Z
4       2014-01-04T23:00:00Z
                ...         
4272    2025-09-11T22:00:00Z
4273    2025-09-12T22:00:00Z
4274    2025-09-13T22:00:00Z
4275    2025-09-14T22:00:00Z
4276    2025-09-15T22:00:00Z
Name: datetime_utc, Length: 4277, dtype: object

In [13]:
df_clean["converted_datetime_utc_date"] = pd.to_datetime(df_clean["datetime_utc"], utc=True)

df_clean["converted_datetime_utc_date"]

0      2013-12-31 23:00:00+00:00
1      2014-01-01 23:00:00+00:00
2      2014-01-02 23:00:00+00:00
3      2014-01-03 23:00:00+00:00
4      2014-01-04 23:00:00+00:00
                  ...           
4272   2025-09-11 22:00:00+00:00
4273   2025-09-12 22:00:00+00:00
4274   2025-09-13 22:00:00+00:00
4275   2025-09-14 22:00:00+00:00
4276   2025-09-15 22:00:00+00:00
Name: converted_datetime_utc_date, Length: 4277, dtype: datetime64[ns, UTC]

In [14]:
df_clean_backup = df_clean.copy()

df_clean_backup = df_clean_backup.copy()

In [15]:
df_clean = df_clean.drop(columns = ["datetime", "datetime_utc", "tz_time", "geo_id", "geo_name"])

df_clean

,value,converted_datetime_utc_date
0,3187298.0,2013-12-31 23:00:00+00:00
1,3916726.0,2014-01-01 23:00:00+00:00
2,3957472.0,2014-01-02 23:00:00+00:00
3,3726719.0,2014-01-03 23:00:00+00:00
4,3511386.0,2014-01-04 23:00:00+00:00
...,...,...
4272,7719284.0,2025-09-11 22:00:00+00:00
4273,6931088.0,2025-09-12 22:00:00+00:00
4274,6490902.0,2025-09-13 22:00:00+00:00
4275,3386155.0,2025-09-14 22:00:00+00:00


In [16]:
#hacemos columandas de dias anteriores 1,3 y 7

df_clean["past_day"]=df_clean["value"].shift(1)
df_clean["past_2day"]=df_clean["value"].shift(2)
df_clean["past_7day"]=df_clean["value"].shift(7)

In [17]:
df_clean["day_week"] = df_clean["converted_datetime_utc_date"].dt.dayofweek    # 0=lunes, 6=domingo
df_clean["day"] = df_clean["converted_datetime_utc_date"].dt.day
df_clean["month"] = df_clean["converted_datetime_utc_date"].dt.month
df_clean["year"] = df_clean["converted_datetime_utc_date"].dt.year

In [18]:
df_clean["nextday"]=df_clean["value"].shift(-1)
df_clean["next2day"]=df_clean["value"].shift(-2)
df_clean["next3day"]=df_clean["value"].shift(-3)


In [19]:
df_testear=df_clean.copy()

In [20]:
df_clean

,value,converted_datetime_utc_date,past_day,past_2day,past_7day,day_week,day,month,year,nextday,next2day,next3day
0,3187298.0,2013-12-31 23:00:00+00:00,NaN,NaN,NaN,1,31,12,2013,3916726.0,3957472.0,3726719.0
1,3916726.0,2014-01-01 23:00:00+00:00,3187298.0,NaN,NaN,2,1,1,2014,3957472.0,3726719.0,3511386.0
2,3957472.0,2014-01-02 23:00:00+00:00,3916726.0,3187298.0,NaN,3,2,1,2014,3726719.0,3511386.0,3309795.0
3,3726719.0,2014-01-03 23:00:00+00:00,3957472.0,3916726.0,NaN,4,3,1,2014,3511386.0,3309795.0,4153137.0
4,3511386.0,2014-01-04 23:00:00+00:00,3726719.0,3957472.0,NaN,5,4,1,2014,3309795.0,4153137.0,4255484.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,7719284.0,2025-09-11 22:00:00+00:00,7532009.0,7663188.0,7833118.0,3,11,9,2025,6931088.0,6490902.0,3386155.0
4273,6931088.0,2025-09-12 22:00:00+00:00,7719284.0,7532009.0,7004598.0,4,12,9,2025,6490902.0,3386155.0,8521577.0
4274,6490902.0,2025-09-13 22:00:00+00:00,6931088.0,7719284.0,6757812.0,5,13,9,2025,3386155.0,8521577.0,NaN
4275,3386155.0,2025-09-14 22:00:00+00:00,6490902.0,6931088.0,7858303.0,6,14,9,2025,8521577.0,NaN,NaN


In [21]:
df_clean=df_clean.dropna()

In [22]:
df_clean

,value,converted_datetime_utc_date,past_day,past_2day,past_7day,day_week,day,month,year,nextday,next2day,next3day
7,4255484.0,2014-01-07 23:00:00+00:00,4153137.0,3309795.0,3187298.0,1,7,1,2014,4298819.0,4292097.0,3867730.0
8,4298819.0,2014-01-08 23:00:00+00:00,4255484.0,4153137.0,3916726.0,2,8,1,2014,4292097.0,3867730.0,3601448.0
9,4292097.0,2014-01-09 23:00:00+00:00,4298819.0,4255484.0,3957472.0,3,9,1,2014,3867730.0,3601448.0,4334742.0
10,3867730.0,2014-01-10 23:00:00+00:00,4292097.0,4298819.0,3726719.0,4,10,1,2014,3601448.0,4334742.0,4433714.0
11,3601448.0,2014-01-11 23:00:00+00:00,3867730.0,4292097.0,3511386.0,5,11,1,2014,4334742.0,4433714.0,4398526.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,7776280.0,2025-09-08 22:00:00+00:00,7858303.0,6757812.0,7496838.0,0,8,9,2025,7663188.0,7532009.0,7719284.0
4270,7663188.0,2025-09-09 22:00:00+00:00,7776280.0,7858303.0,7745092.0,1,9,9,2025,7532009.0,7719284.0,6931088.0
4271,7532009.0,2025-09-10 22:00:00+00:00,7663188.0,7776280.0,7899120.0,2,10,9,2025,7719284.0,6931088.0,6490902.0
4272,7719284.0,2025-09-11 22:00:00+00:00,7532009.0,7663188.0,7833118.0,3,11,9,2025,6931088.0,6490902.0,3386155.0


In [23]:
corte  = df_clean["converted_datetime_utc_date"].max() - pd.Timedelta(days=60)

test_df = df_clean[df_clean["converted_datetime_utc_date"] >= corte]
train_df = df_clean[df_clean["converted_datetime_utc_date"] <  corte]

print(f"Train: \n Fecha min: {train_df["converted_datetime_utc_date"].min()} \n Fecha max: {train_df["converted_datetime_utc_date"].max()} \n Len: {len(train_df)}")
print(f"Test: \n Fecha min: {test_df["converted_datetime_utc_date"].min()} \n Fecha max: {test_df["converted_datetime_utc_date"].max()} \n Len: {len(test_df)}")

Train: 
 Fecha min: 2014-01-07 23:00:00+00:00 
 Fecha max: 2025-07-13 22:00:00+00:00 
 Len: 4206
Test: 
 Fecha min: 2025-07-14 22:00:00+00:00 
 Fecha max: 2025-09-12 22:00:00+00:00 
 Len: 61


In [24]:
X_train=train_df[["value","past_day","past_2day","past_7day","day_week","day","month","year"]]
y_train=train_df["nextday"]
y_train2=train_df["next2day"]
y_train3=train_df["next3day"]
X_test=test_df[["value","past_day","past_2day","past_7day","day_week","day","month","year"]]
y_test=test_df["nextday"]
y_test2=test_df["next2day"]
y_test3=test_df["next3day"]


In [25]:
print(X_train)


          value   past_day  past_2day  past_7day  day_week  day  month  year
7     4255484.0  4153137.0  3309795.0  3187298.0         1    7      1  2014
8     4298819.0  4255484.0  4153137.0  3916726.0         2    8      1  2014
9     4292097.0  4298819.0  4255484.0  3957472.0         3    9      1  2014
10    3867730.0  4292097.0  4298819.0  3726719.0         4   10      1  2014
11    3601448.0  3867730.0  4292097.0  3511386.0         5   11      1  2014
...         ...        ...        ...        ...       ...  ...    ...   ...
4208  8705226.0  8742427.0  8484419.0  9245079.0         2    9      7  2025
4209  8460125.0  8705226.0  8742427.0  9069531.0         3   10      7  2025
4210  7140880.0  8460125.0  8705226.0  8000435.0         4   11      7  2025
4211  6469054.0  7140880.0  8460125.0  7337905.0         5   12      7  2025
4212  8162749.0  6469054.0  7140880.0  8592305.0         6   13      7  2025

[4206 rows x 8 columns]


In [26]:
print(X_test)

          value   past_day  past_2day  past_7day  day_week  day  month  year
4213  8523470.0  8162749.0  6469054.0  8484419.0         0   14      7  2025
4214  8789639.0  8523470.0  8162749.0  8742427.0         1   15      7  2025
4215  9041515.0  8789639.0  8523470.0  8705226.0         2   16      7  2025
4216  8809219.0  9041515.0  8789639.0  8460125.0         3   17      7  2025
4217  7477029.0  8809219.0  9041515.0  7140880.0         4   18      7  2025
...         ...        ...        ...        ...       ...  ...    ...   ...
4269  7776280.0  7858303.0  6757812.0  7496838.0         0    8      9  2025
4270  7663188.0  7776280.0  7858303.0  7745092.0         1    9      9  2025
4271  7532009.0  7663188.0  7776280.0  7899120.0         2   10      9  2025
4272  7719284.0  7532009.0  7663188.0  7833118.0         3   11      9  2025
4273  6931088.0  7719284.0  7532009.0  7004598.0         4   12      9  2025

[61 rows x 8 columns]


In [27]:
print(y_train)

7       4298819.0
8       4292097.0
9       3867730.0
10      3601448.0
11      4334742.0
          ...    
4208    8460125.0
4209    7140880.0
4210    6469054.0
4211    8162749.0
4212    8523470.0
Name: nextday, Length: 4206, dtype: float64


In [28]:
print(y_test)

4213    8789639.0
4214    9041515.0
4215    8809219.0
4216    7477029.0
4217    6847849.0
          ...    
4269    7663188.0
4270    7532009.0
4271    7719284.0
4272    6931088.0
4273    6490902.0
Name: nextday, Length: 61, dtype: float64


In [29]:
#para un dia
model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8
)

model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [30]:
#para 2 dias
model2 = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8
)

model2.fit(X_train,y_train2,eval_set=[(X_test,y_test2)],verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [31]:
#para 3 dias
model3 = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8
)

model3.fit(X_train,y_train3,eval_set=[(X_test,y_test3)],verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, ...)

In [32]:
prediccion=df_testear.iloc[-1:][["value","past_day","past_2day","past_7day","day_week","day","month","year"]]
nextday=model.predict(prediccion)[0]
y_pred=model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"la prediccion para un dia seria: {nextday} con el MAE: {mae}")

la prediccion para un dia seria: 8479251.0 con el MAE: 199712.40983606558


In [33]:
prediccion=df_testear.iloc[-1:][["value","past_day","past_2day","past_7day","day_week","day","month","year"]]
next2days=model2.predict(prediccion)[0]
y_pred2=model2.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"la prediccion para 2 dias seria: {next2days} con el MAE: {mae}")

la prediccion para 2 dias seria: 8092269.0 con el MAE: 199712.40983606558


In [34]:
prediccion=df_testear.iloc[-1:][["value","past_day","past_2day","past_7day","day_week","day","month","year"]]
next3days=model3.predict(prediccion)[0]
y_pred3=model3.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"la prediccion para 3 dias seria: {next3days} con el MAE: {mae}")

la prediccion para 3 dias seria: 7746679.0 con el MAE: 199712.40983606558
